# Imports, setting up device and seeds

In [2]:
import torch
from torch import nn
import torchvision
import torchvision.transforms as transforms
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import numpy as np
 
## PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
import torch.optim as optim

from torch.utils.data import DataLoader

In [3]:
import os
import random

def set_all_seeds(seed):
    os.environ["PL_GLOBAL_SEED"] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

set_all_seeds(42)

In [4]:
# torch.cuda.set_device(1)
device = torch.device("cuda")
# device = torch.device('mps')
# device = torch.device('cpu')

# Constants

In [5]:
VALIDATION_PERCENTAGE = 0.10
TRAIN_PATH = "/kaggle/input/traffic/trafic_32"

# Loading data

In [6]:
%%time
transform = transforms.Compose([transforms.ToTensor()])

batch_size = 256

trainFolder = torchvision.datasets.ImageFolder(root=TRAIN_PATH,
                                               transform=transform)

trainloader = torch.utils.data.DataLoader(trainFolder, batch_size=batch_size, drop_last=True,
                                          shuffle=True, num_workers=2)

# testset = torchvision.datasets.CIFAR10(root=TEST_PATH, train=False,
#                                        download=False, transform=transform)

# testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
#                                          shuffle=False, num_workers=2)


CPU times: user 1.78 s, sys: 3.42 s, total: 5.2 s
Wall time: 3min 11s


In [7]:
trainFolder

Dataset ImageFolder
    Number of datapoints: 39209
    Root location: /kaggle/input/traffic/trafic_32
    StandardTransform
Transform: Compose(
               ToTensor()
           )

# Model

In [8]:
class Encoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, latent_dim):
        super(Encoder, self).__init__()

        self.fc_1 = nn.Linear(input_dim, hidden_dim)
        self.fc_2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc_mean  = nn.Linear(hidden_dim, latent_dim)
        self.fc_var   = nn.Linear(hidden_dim, latent_dim)
        
        self.LeakyReLU = nn.LeakyReLU(0.2)
        
        self.training = True
        
    def forward(self, x):
        x = torch.flatten(x, 1)
        x       = self.LeakyReLU(self.fc_1(x))
        x       = self.LeakyReLU(self.fc_2(x))
        mean     = self.fc_mean(x)
        log_var  = self.fc_var(x)                      # encoder produces mean and log of variance 
                                                       #             (i.e., parateters of simple tractable normal distribution "q"
        
        return mean, log_var

In [9]:
class Decoder(nn.Module):
    def __init__(self, latent_dim, hidden_dim, output_dim):
        super(Decoder, self).__init__()
        self.fc_1 = nn.Linear(latent_dim, hidden_dim)
        self.fc_2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc_3 = nn.Linear(hidden_dim, output_dim)
        
        self.LeakyReLU = nn.LeakyReLU(0.2)
        
    def forward(self, x):
        h     = self.LeakyReLU(self.fc_1(x))
        h     = self.LeakyReLU(self.fc_2(h))
        
        x_hat = torch.sigmoid(self.fc_3(h))
        x_hat = x_hat.view([-1, 3, 32, 32])
        return x_hat

In [10]:
class VAE(nn.Module):
    def __init__(self, x_dim, hidden_dim, latent_dim):
        super(VAE, self).__init__()
        self.latent_dim = latent_dim
        self.encoder = Encoder(input_dim=x_dim, hidden_dim=hidden_dim, latent_dim=latent_dim)
        self.decoder = Decoder(latent_dim=latent_dim, hidden_dim = hidden_dim, output_dim = x_dim)

        
    def reparameterization(self, mean, var):
        eps = torch.randn_like(mean)
        z = eps * var + mean
        # z = np.random.normal(mean, var)
        # z = mean # Change to proper sampling
        return z
        
                
    def forward(self, x):
        mean, log_var = self.encoder(x)
#         print("mean log_var", mean.shape, log_var.shape)
        z = self.reparameterization(mean, torch.exp(0.5 * log_var)) # takes exponential function (log var -> var)
#         print("z", z.shape)
        x_hat = self.decoder(z)
        return x_hat, mean, log_var

In [11]:
vae = VAE(latent_dim=32, hidden_dim=1024, x_dim=32*32*3).to(device)

# Training loop

In [12]:
criterion = nn.MSELoss(reduction="sum")
optimizer = optim.Adam(vae.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.ExponentialLR(optimizer=optimizer, gamma=0.99)

In [13]:
num_epochs = 30
for n in range(num_epochs):
    losses_epoch = []
    for x, _ in iter(trainloader):
        x = x.to(device)
#         print("X SHAPE: ", x.shape, x.shape[0])
        out, means, log_var = vae(x)
        out = out.cpu()
        x = x.cpu()
#         out = x.view (x.shape[0],3,32,32)
#         print("OUT SHAPE: ", out.shape)
#         print("X SHAPE: ", x.shape)
        loss = criterion(out, x) 
        losses_epoch.append(loss.item())
        loss.backward()               
        optimizer.step()             
        optimizer.zero_grad()  
    L1_list = []
#     if n % 10 == 0:
    # for x, _ in iter(test_loader):
    #     x  = x.to(device)
    #     out, _, _ = vae(x)
    #     L1_list.append(torch.mean(torch.abs(out-x)).item())
    print(f"Epoch {n} loss {np.mean(np.array(losses_epoch))}, test L1 = {np.mean(L1_list)}")
    scheduler.step()

Epoch 0 loss 28115.556040645424, test L1 = nan


/opt/conda/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3441: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/opt/conda/lib/python3.7/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Epoch 1 loss 15353.285130718954, test L1 = nan
Epoch 2 loss 12372.034466911764, test L1 = nan
Epoch 3 loss 9969.225790185867, test L1 = nan
Epoch 4 loss 8747.529778773489, test L1 = nan
Epoch 5 loss 7891.340577767565, test L1 = nan
Epoch 6 loss 7317.474175347223, test L1 = nan
Epoch 7 loss 6920.471797768587, test L1 = nan
Epoch 8 loss 6445.970677593954, test L1 = nan
Epoch 9 loss 6116.051087622549, test L1 = nan
Epoch 10 loss 5802.055941712622, test L1 = nan
Epoch 11 loss 5587.338442733864, test L1 = nan
Epoch 12 loss 5434.833843954249, test L1 = nan
Epoch 13 loss 5322.864372702206, test L1 = nan
Epoch 14 loss 5172.91783471201, test L1 = nan
Epoch 15 loss 5027.520724826389, test L1 = nan
Epoch 16 loss 4948.33092064951, test L1 = nan
Epoch 17 loss 4859.142335580065, test L1 = nan
Epoch 18 loss 4815.03764233558, test L1 = nan
Epoch 19 loss 4727.309326171875, test L1 = nan
Epoch 20 loss 4636.450614979065, test L1 = nan
Epoch 21 loss 4601.500327116523, test L1 = nan
Epoch 22 loss 4527.9722